# NMSI–π*–HDQG Simulation Diagnostics
This notebook provides ready-to-use plots and Lyapunov analysis for comparing Classical Navier–Stokes and Augmented NMSI runs.
Load the .npz or .csv outputs from your simulations into this workflow.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')

## Load Data

In [ ]:
# Example: replace with your simulation output paths
data_classical = np.load('out_classical/final_timeseries.npz')
data_augmented = np.load('out_augmented/final_timeseries.npz')

t_c = data_classical['t']
E_c = data_classical['E']
Omega_c = data_classical['Omega']

t_a = data_augmented['t']
E_a = data_augmented['E']
Omega_a = data_augmented['Omega']

## Energy and Enstrophy

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(t_c,E_c,label='Classical')
plt.plot(t_a,E_a,label='Augmented')
plt.xlabel('t'); plt.ylabel('Energy E(t)'); plt.legend()

plt.subplot(1,2,2)
plt.plot(t_c,Omega_c,label='Classical')
plt.plot(t_a,Omega_a,label='Augmented')
plt.xlabel('t'); plt.ylabel('Enstrophy Ω(t)'); plt.legend()

plt.tight_layout(); plt.show()

## Max Vorticity

In [ ]:
if 'wmax' in data_classical and 'wmax' in data_augmented:
    plt.plot(t_c,data_classical['wmax'],label='Classical')
    plt.plot(t_a,data_augmented['wmax'],label='Augmented')
    plt.xlabel('t'); plt.ylabel('‖ω‖∞'); plt.legend(); plt.show()
else:
    print('No wmax field found in data.')

## Energy Spectrum E(k)

In [ ]:
# Example: load spectrum from file (replace with your own)
# spectrum = np.load('out_augmented/spectrum_Ek.npz')
# k = spectrum['k']; Ek = spectrum['Ek']
# plt.loglog(k,Ek,label='Augmented')
# plt.loglog(k,k**(-5/3)*Ek[5]/k[5]**(-5/3),'--',label='-5/3 slope')
# plt.xlabel('k'); plt.ylabel('E(k)'); plt.legend(); plt.show()

## Lyapunov Estimation

In [ ]:
# Load twin trajectory runs (Classical or Augmented)
A = np.load('run_base/final_timeseries.npz')
B = np.load('run_pert/final_timeseries.npz')

t = np.minimum(A['t'], B['t'])
d = np.abs(A['E'][:len(t)] - B['E'][:len(t)]) + 1e-16

i0, i1 = int(0.2*len(t)), int(0.8*len(t))
coef = np.polyfit(t[i0:i1], np.log(d[i0:i1]), 1)
lam = coef[0]

print(f'Estimated λ_max ≈ {lam:.4e}')

plt.plot(t,np.log(d))
plt.axvspan(t[i0],t[i1],alpha=0.2)
plt.xlabel('t'); plt.ylabel('log d(t)')
plt.title(f'Lyapunov slope ~ {lam:.3e}')
plt.show()

### Next Steps
- Adjust ν, grid size, π* and γ parameters.
- Save spectra at multiple times for inertial range analysis.
- Share plots and λ_max results for community review.